# LLM Memory

In chat applications, retaining information from previous interactions is essential to maintain a consistent conversation flow. Let's explore when and how to use different types of memory. 

* [1. Monitoring Message History](#monitoring)
    * [1.1. ConversationChain](#conversation)
    * [1.2. ConversationBufferMemory](#buffer)
    * [1.3. Benefits and Trade-Offs](#benefits)
* [2. Memory Types in LangChain](#memory_types)
    * [2.1. ConversationBufferMemory](#buffer_memory)
    * [2.2. ConversationBufferWindowMemory](#window_memory)
    * [2.3. ConversationSummaryMemory](#summary_memory)
    * [2.4. ConversationSummaryBufferMemory](#summary_buffer_memory)
    * [2.5. Strategies to pick Memory Type](#strategies)
* [3. Chat with a GitHub Repository](#github)
* [4. QA Chatbot over Documents with Sources](#with_sources)
* [5. QA on Financial Data](#financial)
* [6. QA on Any Data](#any)
* [7. Additional Sources](#sources)

In [1]:
import os
from keys import OPENAI_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

<hr>
<a class="anchor" id="monitoring">
    
## 1. Monitoring Message History
    
</a>

<hr>
<a class="anchor" id="conversation">
    
### 1.1. ConversationChain
    
</a>

In [2]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(model_name="text-davinci-003", temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

output = conversation.predict(input="Hi there!")
print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.
 Hi there! It's nice to meet you. How can I help you today?


In [3]:
output = conversation.predict(input="In what scenarios extra memory should be used?")
output = conversation.predict(input="There are various types of memory in Langchain. When to use which type?")
output = conversation.predict(input="Do you remember what was our first message?")

print(output)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: In what scenarios extra memory should be used?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. How can I help you today?
Human: In what scenarios extra memory should be used?
AI:  Extra memory should be used when you need to store more 

As can be noticed from the “Current Conversation” section of the output, the model have access to all the previous messages. It can also remember what the initial message were after 3 questions.

<hr>
<a class="anchor" id="buffer">
    
### 1.2. ConversationBufferMemory
    
</a>

To provide a history of messages (keep context), the `ConversationChain` uses the `ConversationBufferMemory` class by default. This memory can save the previous conversations in form of variables. The class accepts the `return_messages` argument which is helpful for dealing with chat models. 

In [4]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi there!"}, {"output": "Hi there! It's nice to meet you. How can I help you today?"})

print( memory.load_memory_variables({}) )

{'history': [HumanMessage(content='hi there!', additional_kwargs={}, example=False), AIMessage(content="Hi there! It's nice to meet you. How can I help you today?", additional_kwargs={}, example=False)]}


In [5]:
# Alternative code, which will automatically call the .save_context() object after each interaction
from langchain.chains import ConversationChain

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

In [6]:
# Example 1 
# Usage of "ConversationChain" and "ConversationBufferMemory"
from langchain import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)


print( conversation.predict(input="Tell me a joke about elephants") )
print( conversation.predict(input="Who is the author of the Harry Potter series?") )
print( conversation.predict(input="What was the joke you told me earlier?") )

.

AI: What did the elephant say to the naked man? "How do you breathe through that tiny thing?"

AI: The author of the Harry Potter series is J.K. Rowling.

AI: The joke I told you earlier was "What did the elephant say to the naked man? 'How do you breathe through that tiny thing?'"


In [7]:
# Example 2 
# Usage of "ConversationChain" and "ConversationBufferMemory"
from langchain import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm, verbose=True)

# Start with general question
user_message = "Tell me about the history of the Internet."
response = conversation(user_message)
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Tell me about the history of the Internet.

> Finished chain.
{'input': 'Tell me about the history of the Internet.', 'history': [HumanMessage(content='Tell me about the history of the Internet.', additional_kwargs={}, example=False), AIMessage(content='\n\nAI: The Internet has a long and complex history. It began in the 1960s as a project of the United States Department of Defense, which wanted to create a network of computers that could communicate with each other. This network eventually became known as ARPANET, and it was the first example of what we now call the Internet. In the 1980s, the National Science Foundation created the NSFNET, which connected universities and research centers across the United States. This network eventually became the backbone of the modern Internet. In the 1990s, the World Wide Web was created, which al

In [8]:
# User sends another message
user_message = "Who are some important figures in its development?"
response = conversation(user_message)
print(response)  # Chatbot responds, recalling the previous topic and interpriting "its"



> Entering new ConversationChain chain...
Prompt after formatting:
System: The following is a friendly conversation between a human and an AI.
Human: Tell me about the history of the Internet.
AI: 

AI: The Internet has a long and complex history. It began in the 1960s as a project of the United States Department of Defense, which wanted to create a network of computers that could communicate with each other. This network eventually became known as ARPANET, and it was the first example of what we now call the Internet. In the 1980s, the National Science Foundation created the NSFNET, which connected universities and research centers across the United States. This network eventually became the backbone of the modern Internet. In the 1990s, the World Wide Web was created, which allowed people to access information from all over the world. Since then, the Internet has grown exponentially, and it is now used by billions of people around the world.
Human: Who are some important figures in

<hr>
<a class="anchor" id="benefits">
    
### 1.3. Benefits and Trade-Offs
    
</a>

Keeping track of message history in chatbot interactions yields several **benefits**:
- chatbot gains a stronger <font color=green>**sense of context**</font> from previous interactions, improving the accuracy and relevance of its responses;
- the recorded history is a valuable <font color=green>**resource for troubleshooting**</font>, which traces the sequence of events potentially helpful to identify issues;
- effective monitoring systems that include log tracking can <font color=green>**trigger notifications**</font> based on alert conditions, aiding in the early detection of conversation anomalies;
- monitoring message history provides a means to <font color=green>**evaluate**</font> the chatbot's performance over time, paving the way for necessary adjustments and enhancements.

There are also **trade-offs** to consider:
- Storing extensive message history can lead to significant <font color=orange>**memory and storage**</font> usage, potentially impacting the overall system performance;
- Maintaining conversation history might present <font color=orange>**privacy issues**</font>, particularly when sensitive or personally identifiable information is involved, so it is crucial to manage such data with utmost responsibility and in compliance with the relevant data protection regulations.

<hr>
<a class="anchor" id="memory_types">
    
## 2. Memory Types in LangChain
    
</a>

By default, LLMs are stateless, meaning they process each incoming query in isolation, without considering previous interactions. To overcome this limitation, LangChain offers a standard interface for memory, a variety of memory implementations, and examples of chains and agents that employ memory. 

There are several types of Conversational Memory implementations, each with its own advantages and disadvantages:
*************************
- **ConversationBufferMemory**: the most straightforward - stores the entire conversation history as a single string; 
- **ConversationBufferWindowMemory**: maintains a memory window that keeps a limited number of past interactions based on the specified window size;
- **ConversationSummaryMemory**: the most complex variant of memory that holds a summary of previous converations;
- **ConversationSummaryBufferMemory**: keeps a buffer of recent interactions in memory and compiles old interactions into a summary. 
*************************

<hr>
<a class="anchor" id="buffer_memory">
    
### 2.1. ConversationBufferMemory
    
</a>

Advantages:
- maintains a complete record of  the conversation;
- straightforward to implement and use;

Disadvantages:
- may lead to excessive repetition as the conversation grows;
- can become too long for the model's token limit (if the model's token limit is surpassed, the buffer gets truncated, as a result, the conversation context might lose some older information).

In [11]:
from langchain.memory import ConversationBufferMemory
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI(model_name="text-davinci-003", temperature=0)

conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)
conversation.predict(input="Hello!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hello!
AI:

> Finished chain.


" Hi there! It's nice to meet you. How can I help you today?"

In [14]:
print(conversation.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


In [15]:
# The code below creates a prompt template for the customer support chatbot
from langchain import OpenAI, LLMChain, PromptTemplate
from langchain.memory import ConversationBufferMemory

template = """You are a customer support chatbot for a highly advanced customer support AI 
for an online store called "Galactic Emporium," which specializes in selling unique,
otherworldly items sourced from across the universe. You are equipped with an extensive
knowledge of the store's inventory and possess a deep understanding of interstellar cultures. 
As you interact with customers, you help them with their inquiries about these extraordinary
products, while also sharing fascinating stories and facts about the cosmos they come from.

{chat_history}
Customer: {customer_input}
Support Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "customer_input"], 
    template=template
)
chat_history=""

convo_buffer = ConversationChain(
    llm=llm,
    memory=ConversationBufferMemory()
)

In [16]:
# Call the chatbot multiple times to imitate a user’s interaction
convo_buffer("I'm interested in buying items from your store")
convo_buffer("I want toys for my pet, do you have those?")
convo_buffer("I'm interested in price of a chew toys, please")

{'input': "I'm interested in price of a chew toys, please",
 'history': "Human: I'm interested in buying items from your store\nAI:  Great! We have a wide selection of items available for purchase. What type of items are you looking for?\nHuman: I want toys for my pet, do you have those?\nAI:  Yes, we do! We have a variety of pet toys, including chew toys, interactive toys, and plush toys. Do you have a specific type of toy in mind?",
 'response': " Sure! We have a range of chew toys available, with prices ranging from $5 to $20. Is there a particular type of chew toy you're interested in?"}

**Note**: The cost of utilizing the AI model in ConversationBufferMemory is directly influenced by the number of tokens used in a conversation, thereby impacting the overall expenses. LLMs like ChatGPT have token limits, and the more tokens used, the more expensive the API requests become.

To calculate token count in a conversation, you can use the `tiktoken` package.

In [17]:
# Using "tiktoken" to calculate token count in a conversation
import tiktoken

def count_tokens(text: str) -> int:
    tokenizer = tiktoken.encoding_for_model("gpt-3.5-turbo")
    tokens = tokenizer.encode(text)
    return len(tokens)

conversation = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
]

total_tokens = 0
for message in conversation:
    total_tokens += count_tokens(message["content"])

print(f"Total tokens in the conversation: {total_tokens}")

Total tokens in the conversation: 29


<hr>
<a class="anchor" id="window_memory">
    
### 2.2. ConversationBufferWindowMemory
    
</a>

Advantages:
- more efficient than ConversationBufferMemory, because stores a fixed number of recent messages in the conversation;
- low risk of exceeding the model's token limit; 

Disadvantages:
- does not maintain the complete conversation history (so, the chatbot can lose context if essential information falls outside the fixed window of messages).

In [18]:
# Create a prompt template for the tour guide chatbot
from langchain.memory import ConversationBufferWindowMemory
from langchain import OpenAI, LLMChain, PromptTemplate

template = """You are a cutting-edge virtual tour guide for
 an art gallery that showcases masterpieces from alternate dimensions and
 timelines. Your advanced AI capabilities allow you to perceive and understand
 the intricacies of each artwork, as well as their origins and significance in
 their respective dimensions. As visitors embark on their journey with you
 through the gallery, you weave enthralling tales about the alternate histories
 and cultures that gave birth to these otherworldly creations.

{chat_history}
Visitor: {visitor_input}
Tour Guide:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "visitor_input"], 
    template=template
)

chat_history=""

convo_buffer_win = ConversationChain(
    llm=llm,
    memory = ConversationBufferWindowMemory(k=3, return_messages=True)
)

The value of `k` represents the number of past messages to be stored in the buffer. The `return_messages` parameter, when set to True, indicates that the stored messages should be returned when the memory is accessed. 

In [19]:
# A sample conversation with the chatbot
convo_buffer_win("What is your name?")
convo_buffer_win("What can you do?")
convo_buffer_win("Do you mind give me a tour, I want to see your galery?")
convo_buffer_win("what is your working hours?")
convo_buffer_win("See you soon.")

{'input': 'See you soon.',
 'history': [HumanMessage(content='What can you do?', additional_kwargs={}, example=False),
  AIMessage(content=" I can help you with a variety of tasks. I can answer questions, provide information, and even help you with research. I'm also capable of learning new things, so I'm always expanding my capabilities.", additional_kwargs={}, example=False),
  HumanMessage(content='Do you mind give me a tour, I want to see your galery?', additional_kwargs={}, example=False),
  AIMessage(content=" Sure! I'd be happy to give you a tour of my gallery. I have a variety of images, videos, and other media that I can show you. Let me know if there's anything specific you'd like to see.", additional_kwargs={}, example=False),
  HumanMessage(content='what is your working hours?', additional_kwargs={}, example=False),
  AIMessage(content=" I'm available 24/7! I'm always here to help you with whatever you need.", additional_kwargs={}, example=False)],
 'response': ' Sure thing

<hr>
<a class="anchor" id="summary_memory">
    
### 2.3. ConversationSummaryMemory
    
</a>


Advantages:

- helps reduce the number of tokens by summarizing the conversation;
- flexibility in configuration (the history can be returned either as a list of messages or as a plain text summary);
- the `predict_new_summary` method allows to directly obtain a summary prediction based on the list of messages and the previous summary;

Disadvantages:

- summarizing the conversation might lead to a loss of information;
- requires more processing to generate the summary, potentially affecting the performance of the chatbot. 

In [21]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryMemory

# Create a ConversationChain with ConversationSummaryMemory
conversation_with_summary = ConversationChain(
    llm=llm, 
    memory=ConversationSummaryMemory(llm=llm),
    verbose=True
)

# Example conversation
response = conversation_with_summary.predict(input="Hi, what's up?")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.
 Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?


<hr>
<a class="anchor" id="summary_buffer_memory">
    
### 2.4. ConversationSummaryBufferMemory
    
</a>

Advantages:

- Ability to remember distant interactions through summarization while keeping recent interactions in their raw, information-rich form;
- Flexible token management allowing to control of the maximum number of tokens used for memory, which can be adjusted based on needs;

Disadvantages:

- Requires more tweaking on what to summarize and what to maintain within the buffer window;
- May still exceed context window limits for very long conversations.

In [22]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["topic"],
    template="The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\nCurrent conversation:\n{topic}",
)

In [24]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory

llm = OpenAI(temperature=0)
conversation_with_summary = ConversationChain(
    llm=llm,
    memory=ConversationSummaryBufferMemory(llm=OpenAI(), max_token_limit=40),
    verbose=True
)

conversation_with_summary.predict(input="Hi, what's up?")
conversation_with_summary.predict(input="Just working on writing some documentation!")
response = conversation_with_summary.predict(input="For LangChain! Have you heard of it?")
print(response)



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: Just working on writing some documentation!
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversatio

<hr>
<a class="anchor" id="strategies">
    
### 2.5. Strategies to pick Memory Type
    
</a>

If the `ConversationBufferMemory` surpasses the token limit of the model, then the model will not be able to handle the conversation (an error will be received). There are different strategies, to manage this situation:

1. Remove oldest messages: remove the oldest messages in the conversation transcript once the token count is reached. (this method can cause the conversation quality to degrade over time).
2. Limit conversation duration: limit the conversation duration to the max token length or a certain number of turns. 
3. ConversationBufferWindowMemory Method: limit the number of tokens being used by maintaining a fixed-size buffer window that stores only the most recent tokens, up to a specified limit. 
4. ConversationSummaryBufferMemory Approach: summarize the earliest interactions in a conversation and maintain the most recent tokens in their raw, information-rich form, up to a specified limit.

<hr>
<a class="anchor" id="github">
    
## 3. Chat with a GitHub Repository
    
</a>

<hr>
<a class="anchor" id="with_sources">
    
## 4. QA Chatbot over Documents with Sources
    
</a>

<hr>
<a class="anchor" id="financial">
    
## 5. QA on Financial Data
    
</a>

<hr>
<a class="anchor" id="any">
    
## 6. QA on Any Data
    
</a>

<hr>
<a class="anchor" id="sources">
    
## 7. Additional Sources
    
</a>

- [Breaking the Token Limit: How to Work with Large Amounts of Text in ChatGPT](https://marco-gonzalez.medium.com/breaking-the-token-limit-how-to-work-with-large-amounts-of-text-in-chatgpt-da18c798d882)